In [2]:
from time import sleep
from ctypes import c_int, byref
import libximc as ximc
import serial
import pandas
import numpy
from IPython import display
import matplotlib.pyplot as plt

def open_serial(port, baudrate=None):
    if baudrate == None:
        try:
            return serial.Serial(port)
        except Exception as e:
            print(e)
            return None
    else:
        try:
            return serial.Serial(port, baudrate)
        except Exception as e:
            print(e)
            return None
        
def close_serial(*devices):
    for device in devices:
        try:
            device.close()
        except Exception as e:
            print(e)

def read_power(powermeter_device, TEST=False):
    if TEST:
        # заглушка для тестирования
        return np.random.random()
    else:
        try:
            powermeter_device.write(b'*CVU')
            return powermeter_device.readline().decode().strip()
            # print(powermeter_device.readline().decode().strip())
        except Exception as e:
            print(e)
            return None

In [3]:
maestro = open_serial('COM5', 115200)

print(read_power(maestro))

close_serial(maestro)

5.58608e-007


In [4]:
device_uri = "xi-com:\\\\.\\COM3"
device_id = ximc.lib.open_device(device_uri.encode())
if device_id > 0:
    print("Device with URI {} successfully opened".format(device_uri))
else:
    raise RuntimeError("Failed to open device with URI", device_uri)
print("Disconnect device")

ximc.lib.command_movr(device_id, 100, 0)

ximc.lib.close_device(byref(c_int(device_id)))

Device with URI xi-com:\\.\COM3 successfully opened
Disconnect device


0

In [10]:
data = pandas.DataFrame(columns = ['angle', 'power'])

for i in range(1, 148):

    device_uri = "xi-com:\\\\.\\COM3"
    device_id = ximc.lib.open_device(device_uri.encode())
    ximc.lib.command_movr(device_id, 100, 0)
    data.loc[i, 'angle'] = i * 1.22
    ximc.lib.close_device(byref(c_int(device_id)))

    sleep(1)

    maestro = open_serial('COM4', 115200)
    data.loc[i, 'power'] = read_power(maestro)
    close_serial(maestro)

    print(i)

print("Stop movement")
ximc.lib.command_stop(device_id)

now = datetime.datetime.now()
filename = 'result_{}.csv'.format(now.strftime('%Y-%m-%d_%H-%M-%S'))
data.to_csv(filename, index=False)

In [43]:
ximc.lib.close_device(byref(c_int(device_id)))

0

In [36]:
import libximc as ximc
device_uri = "xi-com:\\\\.\\COM3"
device_id = ximc.lib.open_device(device_uri.encode())
if device_id > 0:
    print('ok')
else:
    raise RuntimeError("Failed to open device with URI", device_uri)

ximc.lib.test_left(device_id)

ximc.lib.close_device(byref(c_int(device_id)))

ok


AttributeError: function 'test_left' not found